# Automation
**Prompting Guidelines**
- sampling 75% of personas
- 2 calls per persona
  
__ChatGPT 3.5 Turbo__
- input: \$0.50/1M tokens
- output: \$1.50/1M tokens

__Claude Sonnet/Haiku__
- input: \$0.25/1M tokens
- output: \$1.25/1M tokens

In [2]:
import pandas as pd

df = pd.read_csv('demographic_personas.csv')
sampled_df = df.sample(frac=0.75, random_state=42)  # Set random_state for reproducibility
print("Number of rows in sampled DataFrame:", sampled_df.shape[0])

Number of rows in sampled DataFrame: 688


In [3]:
gsam_prompt = """
The person comes across a stranger who is sitting slumped in a doorway head down, eyes closed, not moving. 
The person hears the stranger coughing and groaning.
"""
q = """
Please provide a value from 0-100 on the person’s inclination to help the stranger. 
0 indicates the person is not at all likely to help to stranger, while 100 indicates the person is very likely to help the stranger.

If any, provide a value for how many minutes the person stays and helps the stranger. 

If any, write a brief story about what actions the person takes.

Please format your answer like so:
Helpfulness: x (integer between 0-100)
Minutes waited: y (integer value)
Actions:
- Person does/does not z.
"""

## Claude

In [ ]:
from anthropic import Anthropic
import os
import json
import time
import pandas as pd

claude_df = pd.read_csv('claude_responses.csv')
# print(claude_df)

delay = 15

def makeApiCall(prompt, num):
    auth_pers = json.loads(open("api_auth.txt").read()) # add api auth
    os.environ["ANTHROPIC_API_KEY"] = auth_pers['anthropic']['secretKey']
    client = Anthropic()
    
    full_context = prompt + gsam_prompt + q + num
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=300,
        temperature=0.0,
        system="You are a helpful observer who can provide insight on people.",
        messages=[
            {"role": "user", "content": full_context}
        ]
    )
    return message.content[0].text


for index, row in claude_df.iterrows():
    prompt = row['Prompt'] # get prompt
    print(prompt)
    # print(row['Response 1'])
    # print(row['Response 2'])
    
    if pd.isna(row['Response 1']):
        response1 = makeApiCall(prompt, " (1)")
        # print(response1)
        claude_df.at[index, 'Response 1'] = response1
        print("made call 1")
        time.sleep(delay)
    
    if pd.isna(row['Response 2']):
        response2 = makeApiCall(prompt, " (2)")
        # print(response2)
        claude_df.at[index, 'Response 2'] = response2
        print("made call 2")
    
    claude_df.to_csv('claude_responses.csv', index=False)
print(claude_df)

A Hispanic / Latine senior citizen person who identifies as nonbinary is upper class
A Asian young adult person who identifies as nonbinary has a high school graduate
A Pacific Islander adult person who identifies as nonbinary is working class
A Asian adult person who identifies as female is married
A White senior citizen person who identifies as male is married
A Pacific Islander senior citizen person who identifies as male is single
A Hispanic / Latine adult person who identifies as nonbinary is in a rush
A Pacific Islander adult person who identifies as female is upper class
A Hispanic / Latine young adult person who identifies as male has no high school diploma
A Pacific Islander adult person who identifies as male is not in a rush
A Hispanic / Latine senior citizen person who identifies as male is married
A Black adult person who identifies as female is not in a rush
A Hispanic / Latine senior citizen person who identifies as male is working class
A Pacific Islander senior citizen

In [32]:
# IF YOU STOP -- RUN THIS CODE
claude_df.to_csv('claude_responses.csv', index=False)
# print(claude_df)

## ChatGPT